In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress


# Import the OMDB API key
from api_keys import omdb_api_key

In [ ]:
# Data files
netflix_path = "../data/netflix_titles.csv"
disney_path = "../data/disney_plus_titles.csv"
prices_path = "../data/subscription_prices.csv"

# Read the Netflix and Disney+ data
netflix_data = pd.read_csv(netflix_path)
disney_data = pd.read_csv(disney_path)
prices_data = pd.read_csv(prices_path)

In [ ]:
# Create Netflix dataframe and review length
netflix_df = pd.DataFrame(netflix_data)
len(netflix_df)

In [ ]:
# Netflix date_added is the key column for filtering to 2019 - 2021 to align with Disney+ dataset date_added dates
# The following steps trim the "September 24, 2018" dates to remove extra spaces, and converts date_added to date format for filtering
netflix_df['date_added'] = netflix_df['date_added'].str.strip()
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], errors='coerce')

# There are a handful of blank date_added that will impact date filtering
netflix_blank_dates = netflix_df[netflix_df['date_added'].isna()]
len(netflix_blank_dates)

In [ ]:
# Drop rows with blank dates
netflix_df = netflix_df.dropna(subset=['date_added'])

# Verify the number of rows with blank dates after dropping
len(netflix_df[netflix_df['date_added'].isna()])

In [ ]:
# Filter out for dates greater than January 1, 2019
netflix_df = netflix_df.loc[(netflix_df['date_added'] >= '2019-01-01') & (netflix_df['date_added'] <= '2021-12-31')]
len(netflix_df)

In [ ]:
# Create Disney+ dataframe and review length
disney_df = pd.DataFrame(disney_data)
disney_df.head()

In [ ]:
# There are a handful of blank date_added that will impact data
disney_blank_dates = disney_df[disney_df['date_added'].isna()]
len(disney_blank_dates)

In [ ]:
# Drop rows with blank dates
disney_df = disney_df.dropna(subset=['date_added'])

# Verify the number of rows with blank dates after dropping
len(disney_df[disney_df['date_added'].isna()])

In [ ]:
# Create Subscription Price dataframe and review length
prices_df = pd.DataFrame(prices_data)
len(prices_df)

In [ ]:
# Add platform column to each streaming service to maintain association
netflix_df["platform"] = "Netflix"
disney_df["platform"] = "Disney+"

In [ ]:
# Concatenate Netflix and Disney+ dataframes
combined_df = pd.concat([netflix_df, disney_df], ignore_index=True)

In [ ]:
combined_df.dtypes

In [ ]:
# Extract the primary genre from listed_in by pulling the first values within the list
combined_df['listed_in'] = combined_df['listed_in'].str.split(',')
combined_df['primary_genre'] = combined_df['listed_in'].str.get(0)
combined_df.head()

# blank_genre = combined_df[combined_df['primary_genre'].isna()]
# len(blank_genre)

In [ ]:
# Clean dataframe with columns of importance for data analysis
combined_df = combined_df.loc[:, ['platform', 'show_id', 'type', 'title', 'date_added', 'release_year', 'primary_genre']]
combined_df.head()

In [ ]:
# Add empty columns for IMDb metadata to be pulled from API
combined_df['OMDb Rating'] = ''
combined_df['OMDb Votes'] = ''

In [ ]:
# Enter in field value replacements for genre normalization (i.e. Documentaries and Docuseries to Documentary)
len(combined_df['primary_genre'].unique())

In [ ]:
# this is being worked on - could not get api response to work (garrett) 

# Establishing base URL for OMDB API
url = "http://www.omdbapi.com/?"

# Construct the API request URL with the title and API key
params = {
        'apikey': omdb_api_key,
    }

# Loop through each title in the 'title' column of combined_df
for index, row in combined_df.iterrows():
    time.sleep(2) # Add a delay to avoid hitting the API too quickly

    params['t'] = combined_df['title'] # get title from combined_df

    # Establishing base URL for OMDB API
    url = "http://www.omdbapi.com/?"
    
   # Run an API request for each of the titles
    omdb_response = requests.get(url, params=params, verify=False)
    print(omdb_response)

In [ ]:
# another try for api call structure
# Establishing base URL for OMDB API
url = "http://www.omdbapi.com/?"

# Loop through each title in the 'title' column of combined_df
for index, title in enumerate(combined_df['title']):
    time.sleep(2) # Add a delay to avoid hitting the API too quickly
    
    # Construct the API request URL with the title and API key
    params = {
        'apikey': omdb_api_key,
        't': title
    }

    # Run an API request for each of the titles
    try:
        # Parse the JSON and retrieve data
        omdb_response = requests.get(url, params=params, verify=False)
        omdb_data = response.json()
    
    # Parse out OMDB ratings, votes, etc.
        rating = omdb_data[] # review and find correct column
        votes = omdb_data[] # review and find correct column
        
    # Assign OMDB information into combined_df
        combined_df.at[index, "OMDb Rating"] = rating, 
        combined_df.at[index, "OMDb Votes"] = rating 
                             # add in other co
                            })
    
 # If an error is experienced, skip the title
    except:
        print("Title not found. Skipping...")
    pass